<a href="https://colab.research.google.com/github/MaryaMalyshkina/MaryaMalyshkina/blob/main/wink_malyshkina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd

In [22]:
from text_util import apply_text_cleaning

In [25]:
from models.calls import CallClassifier
from models.brands import BrandClassifier

import tags
import conquerors
import materials
import employees

In [26]:
model_path = '/content/models/full_model_components.joblib'
mlb_path = '/content/models/mlb_encoder.joblib'

classifier = CallClassifier()
classifier.model_load(model_path, mlb_path)

✅ Модель успешно загружена
📊 Количество классов: 27
🏷️ Доступные теги: ['Бухгалтерия', 'Высокая цена', 'Долгопрудный склад', 'Доставка', 'Жалобы по качеству', 'Заставская офис', 'Карматех/кармапласт', 'Конкуренты', 'Консультация по бренду', 'Консультация по материалу', 'Нет товара/услуги', 'Нецензурное общение', 'Озон/Вб/иные маркетплейсы', 'Оплата', 'Отсрочка', 'Претензия', 'Разгрузка товара', 'Ростов склад', 'Сайт vink.ru', 'Самовывоз', 'Транспортная компания', 'Фамилии сотрудников', 'Часы работы', 'Шоурум Елино', 'Шушары склад', 'ЭДО', 'Экспресс - доставка']


In [27]:
model_path = '/content/models/full_brands_components.joblib'
mlb_path = '/content/models/mlb_brand_encoder.joblib'

brand_classifier = BrandClassifier()
brand_classifier.model_load(model_path, mlb_path)

✅ Модель успешно загружена
📊 Количество классов: 29
🏷️ Доступные теги: ['Allesti', 'Armlord', 'Coleido', 'Dippa', 'Engplast', 'Evoglas', 'Foletti', 'Harvey', 'Hogomaku', 'Indorian', 'Inopan', 'KDX', 'Karmaplast', 'Lamilama', 'Lepe Pasky', 'Lokipix', 'Loriglas', 'Lucentis', 'Moonral', 'Offspark', 'Pichipanda', 'Plo plo', 'Saviar', 'Vikuflex', 'Vikuglas', 'Vikulen', 'Vikulux', 'Vikupet', 'Vikupor']


In [6]:
# classifier.model_train('/content/звонки.xlsx')

In [28]:
classifier.model_save( 'try', 'mlb_try')

✅ Модель сохранена: try
✅ MultiLabelBinarizer сохранен: mlb_try


In [29]:
classifier.get_model_info()

{'status': 'Модель загружена',
 'num_classes': 27,
 'classes': ['Бухгалтерия',
  'Высокая цена',
  'Долгопрудный склад',
  'Доставка',
  'Жалобы по качеству',
  'Заставская офис',
  'Карматех/кармапласт',
  'Конкуренты',
  'Консультация по бренду',
  'Консультация по материалу',
  'Нет товара/услуги',
  'Нецензурное общение',
  'Озон/Вб/иные маркетплейсы',
  'Оплата',
  'Отсрочка',
  'Претензия',
  'Разгрузка товара',
  'Ростов склад',
  'Сайт vink.ru',
  'Самовывоз',
  'Транспортная компания',
  'Фамилии сотрудников',
  'Часы работы',
  'Шоурум Елино',
  'Шушары склад',
  'ЭДО',
  'Экспресс - доставка'],
 'pipeline_steps': ['preprocessor', 'classifier']}

In [30]:
df_calls = pd.read_excel('/content/data/звонки.xlsx', engine='openpyxl')

In [31]:
df_calls.columns

Index(['call_date', 'call_text', 'tags', 'classes'], dtype='object')

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_test, y_train, y_test = train_test_split(
     df_calls.drop(['call_date', 'tags', 'classes'], axis=1),
     df_calls['tags'], test_size=0.2, random_state=42
)

In [34]:
y_test = tags.tags_preprocess(y_test)

In [35]:
text = df_calls['call_text'][0]

In [36]:
classifier.evaluate_model(X_test, y_test)

0.8379621649449177

In [37]:
predictions_df = classifier.get_prediction_dataframe(X_test, threshold=0.5)

In [38]:
predictions_df.head(2)

,Бухгалтерия,Высокая цена,Долгопрудный склад,Доставка,Жалобы по качеству,Заставская офис,Карматех/кармапласт,Конкуренты,Консультация по бренду,Консультация по материалу,...,Ростов склад,Сайт vink.ru,Самовывоз,Транспортная компания,Фамилии сотрудников,Часы работы,Шоурум Елино,Шушары склад,ЭДО,Экспресс - доставка
1222,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
111,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0


In [39]:
classifier.final_result(X_test[:30], brand_classifier,  thresh=0.3)

Разговор 0
    Консультация по бренду 1.0
        Бренды:  [('Saviar',)]
    Оплата 1.0
Разговор 1
    Консультация по бренду 0.99
        Бренды:  [()]
    Оплата 1.0
    Транспортная компания 0.72
Разговор 2
    Оплата 0.99
Разговор 3
    Бухгалтерия 0.99
    Оплата 1.0
Разговор 4
    Бухгалтерия 0.99
    Доставка 0.95
    Консультация по бренду 1.0
        Бренды:  [()]
    Консультация по материалу 1.0
        Материалы:  ['пленки', 'листы']
    Оплата 1.0
    Сайт vink.ru 0.97
Разговор 5
    Оплата 0.99
Разговор 6
    Оплата 1.0
Разговор 7
    Консультация по бренду 1.0
        Бренды:  [()]
    Оплата 1.0
Разговор 8
    Бухгалтерия 1.0
    Доставка 1.0
    Консультация по материалу 0.99
        Материалы:  ['пленки']
    Оплата 1.0
    Разгрузка товара 0.95
    Транспортная компания 0.97
Разговор 9
    Бухгалтерия 0.94
    Оплата 1.0
Разговор 10
    Оплата 1.0
Разговор 11
    Оплата 1.0
    Сайт vink.ru 0.99
    Транспортная компания 0.98
Разговор 12
    Оплата 1.0
    Транспортн

In [19]:
text = '[0.00s -> 10.00s]  Алло. Алло, Юля, добрый день. Да, здравствуй, Андрей. Можно ли заказик накидать?\
Как вы? На когда? [10.00s -> 18.00s]  С водичками и вас. Спасибо. Спасибо и вам. А на завтра\
хотелось бы? Да, давайте, ага. [18.00s -> 32.00s]  Так, что нам, значит, надо? Скажите тогда,\
знаете, с чего с компагита начнём? Чёрный полтора и метр двадцать ноль третий имеются? [32.00s ->\
55.00s]  Сейчас, секундочку. Чёрный только ноль два есть. Ноль два? Ну, полтора на четыре, стенка\
ноль два. Ага, метра двадцать нету, да? Нету. [55.00s -> 72.00s]  Полтора на четыре. Кто это?\
Билдекс. Билдекс. Ну, давайте тогда полтора на четыре, стенка ноль два. Получается, чё, два листа.\
Ага. [72.00s -> 98.00s]  Так, потом белый. А белый? Метр двадцать два есть. Есть. Тогда белого\
полтора метровый один лист. Так, белый полтора метровый один лист. Метр двадцать два листа. Два\
листа. [99.00s -> 110.00s]  Так, получается, по компагиту всё. По компагиту всё. А дальше плёнка для\
печати. Белая матовая, СВР. [110.00s -> 139.00s]  Так, белая матовая. Ага. Так, метр ноль семь.\
Сколько? Один ролик. Метр двадцать семь. Два ролика и ламинация. [140.00s -> 144.00s]  Так,\
ламинация матовая. Метр двадцать семь. Восемьдесят микрон. Два ролика. [145.00s -> 155.00s]  Сейчас,\
секундочку. Так, ламинация матовая. Метр двадцать семь. Восемьдесят микрон. Два ролика. [155.00s ->\
179.00s]  Сколько? Два ролика. Монтажка ноль пятое. Пять миллиметров. [179.00s -> 191.00s]  Пять\
роликов. Так, вроде бы всё. Теперь нам надо намотку. [192.00s -> 204.00s]  Сейчас, одну секундочку.\
Так, доставка на завтра. Завтра десятое. Сейчас, секунду. Я сразу вам счёт отправлю. Вы больше\
ничего не будете сюда добавлять? [205.00s -> 212.00s]  Пока нет. Мы же, если что, можем в течение\
дня там, да, подумать? [213.00s -> 226.00s]  Да, пожалуйста, да. Вы можете сейчас эти заказы\
проверить, подтвердить, а потом, если что, отдельным счётом сделаем. Просто их уже в доставку точно\
поставить. Так, сейчас я это отправлю. И нарезка, да? [227.00s -> 228.00s]  Нарезка, да. [234.00s ->\
239.00s]  А чёрный композит я два, да, заказал? [240.00s -> 245.00s]  Чёрный, два, да, два. [246.00s\
-> 247.00s]  Два, ага, всё верно. [248.00s -> 254.00s]  Так, так, так, ага. Что по нарезке? [254.00s\
-> 259.00s]  Так, шестьсот сорок первое, матовое, метровое, ноль пятьдесят четыре. [272.00s ->\
274.00s]  Так, пятьдесят четвертое, сколько? [275.00s -> 276.00s]  Пять метров. [277.00s -> 278.00s]\
Ага. [278.00s -> 287.00s]  Наверное, всё. Пока на ум ничего не приходит. Сегодня, может быть, ещё в\
течение дня что-нибудь вспомню по нарезке. [288.00s -> 291.00s]  Всё, хорошо, Андрей. Сейчас\
отправляю, тогда проверьте, пожалуйста, заказ подтвердите. [292.00s -> 293.00s]  Всё, ага, спасибо.\
[294.00s -> 295.00s]  Пожалуйста. [295.00s -> 296.00s]  До свидания. [296.00s -> 297.00s]  До'

# classifier.model_predict_single(text)

In [40]:
brand_classifier.model_predict_single_brand(text, threshold=0.5)

[()]